In [1]:
from colabfit.tools.database import MongoDatabase, load_data
from colabfit.tools.property_settings import PropertySettings

client = MongoDatabase('colabfit_rebuild', nprocs=6)

In [2]:
import os
import json
import numpy as np
from ase import Atoms

def reader(file_path, **kwargs):
    with open(file_path) as f:
        data = json.loads('\n'.join(f.readlines()[kwargs['header_lines']:]))

    symbols     = data['Dataset']['Data'][0]['AtomTypes']
    positions   = np.array(data['Dataset']['Data'][0]['Positions'])
    box         = np.array(data['Dataset']['Data'][0]['Lattice'])

    at_name = os.path.splitext(str(file_path).split('JSON')[1][1:])[0]

    try:
        atoms = Atoms(symbols, positions=positions, cell=box, pbc=[1, 1, 1])
    except Exception as e:
        # print("Error on :", at_name, e, set(symbols))
        symbols = symbols[1:]
        atoms = Atoms(symbols, positions=positions, cell=box, pbc=[1, 1, 1])

    atoms.info['name']      = at_name
    atoms.info['energy']    = data['Dataset']['Data'][0]['Energy']
    atoms.arrays['forces']  = np.array(
        data['Dataset']['Data'][0]['Forces']
    )

    atoms.info['stress'] = np.array(data['Dataset']['Data'][0]['Stress'])

    atoms.info['per-atom'] = False
    
    yield atoms

In [ ]:
name = 'WBe_PRB2019'

configurations = list(load_data(
    file_path='/colabfit/data/FitSNAP/examples/WBe_PRB2019/JSON',
    file_format='folder',
    name_field='name',
    elements=['W', 'Be'],
    default_name=name,
    reader=reader,
    glob_string='*.json',
    verbose=True,
    header_lines=1,
))

Loading file 1/25422: 1it [00:00, 1014.10it/s]
Loading file 2/25422: 1it [00:00, 1593.58it/s]
Loading file 3/25422: 1it [00:00, 1931.08it/s]
Loading file 4/25422: 1it [00:00, 1054.64it/s]
Loading file 5/25422: 1it [00:00, 1397.17it/s]
Loading file 6/25422: 1it [00:00, 917.39it/s]
Loading file 7/25422: 1it [00:00, 1071.07it/s]
Loading file 8/25422: 1it [00:00, 997.69it/s]
Loading file 9/25422: 1it [00:00, 1351.26it/s]
Loading file 10/25422: 1it [00:00, 1476.87it/s]
Loading file 11/25422: 1it [00:00, 1146.92it/s]
Loading file 12/25422: 1it [00:00, 1186.51it/s]
Loading file 13/25422: 1it [00:00, 1368.90it/s]
Loading file 14/25422: 1it [00:00, 933.31it/s]
Loading file 15/25422: 1it [00:00, 769.03it/s]
Loading file 16/25422: 1it [00:00, 1331.10it/s]
Loading file 17/25422: 1it [00:00, 520.51it/s]
Loading file 18/25422: 1it [00:00, 343.01it/s]
Loading file 19/25422: 1it [00:00, 186.27it/s]
Loading file 20/25422: 1it [00:00, 877.29it/s]
Loading file 21/25422: 1it [00:00, 864.27it/s]
Loading fi

Loading file 170/25422: 1it [00:00, 1677.72it/s]
Loading file 171/25422: 1it [00:00, 1482.09it/s]
Loading file 172/25422: 1it [00:00, 2312.19it/s]
Loading file 173/25422: 1it [00:00, 2256.22it/s]
Loading file 174/25422: 1it [00:00, 1475.31it/s]
Loading file 175/25422: 1it [00:00, 1421.80it/s]
Loading file 176/25422: 1it [00:00, 1632.02it/s]
Loading file 177/25422: 1it [00:00, 1307.45it/s]
Loading file 178/25422: 1it [00:00, 1705.69it/s]
Loading file 179/25422: 1it [00:00, 2274.57it/s]
Loading file 180/25422: 1it [00:00, 1086.04it/s]
Loading file 181/25422: 1it [00:00, 1465.52it/s]
Loading file 182/25422: 1it [00:00, 1503.33it/s]
Loading file 183/25422: 1it [00:00, 1711.26it/s]
Loading file 184/25422: 1it [00:00, 1702.92it/s]
Loading file 185/25422: 1it [00:00, 1769.75it/s]
Loading file 186/25422: 1it [00:00, 2314.74it/s]
Loading file 187/25422: 1it [00:00, 2207.53it/s]
Loading file 188/25422: 1it [00:00, 1796.28it/s]
Loading file 189/25422: 1it [00:00, 1340.46it/s]
Loading file 190/254

Loading file 337/25422: 1it [00:00, 1287.78it/s]
Loading file 338/25422: 1it [00:00, 2213.35it/s]
Loading file 339/25422: 1it [00:00, 2339.27it/s]
Loading file 340/25422: 1it [00:00, 2251.37it/s]
Loading file 341/25422: 1it [00:00, 2316.02it/s]
Loading file 342/25422: 1it [00:00, 2181.13it/s]
Loading file 343/25422: 1it [00:00, 1307.86it/s]
Loading file 344/25422: 1it [00:00, 1608.25it/s]
Loading file 345/25422: 1it [00:00, 1029.78it/s]
Loading file 346/25422: 1it [00:00, 1635.84it/s]
Loading file 347/25422: 1it [00:00, 2189.09it/s]
Loading file 348/25422: 1it [00:00, 2147.62it/s]
Loading file 349/25422: 1it [00:00, 1546.57it/s]
Loading file 350/25422: 1it [00:00, 1300.16it/s]
Loading file 351/25422: 1it [00:00, 1581.56it/s]
Loading file 352/25422: 1it [00:00, 592.83it/s]
Loading file 353/25422: 1it [00:00, 1669.71it/s]
Loading file 354/25422: 1it [00:00, 1625.07it/s]
Loading file 355/25422: 1it [00:00, 1253.15it/s]
Loading file 356/25422: 1it [00:00, 1378.80it/s]
Loading file 357/2542

Loading file 504/25422: 1it [00:00, 1307.45it/s]
Loading file 505/25422: 1it [00:00, 2347.12it/s]
Loading file 506/25422: 1it [00:00, 1452.82it/s]
Loading file 507/25422: 1it [00:00, 2332.76it/s]
Loading file 508/25422: 1it [00:00, 2279.51it/s]
Loading file 509/25422: 1it [00:00, 188.68it/s]
Loading file 510/25422: 1it [00:00, 1422.76it/s]
Loading file 511/25422: 1it [00:00, 2187.95it/s]
Loading file 512/25422: 1it [00:00, 2220.38it/s]
Loading file 513/25422: 1it [00:00, 1550.57it/s]
Loading file 514/25422: 1it [00:00, 1445.81it/s]
Loading file 515/25422: 1it [00:00, 1323.54it/s]
Loading file 516/25422: 1it [00:00, 2256.22it/s]
Loading file 517/25422: 1it [00:00, 1588.75it/s]
Loading file 518/25422: 1it [00:00, 1358.26it/s]
Loading file 519/25422: 1it [00:00, 1393.46it/s]
Loading file 520/25422: 1it [00:00, 1567.38it/s]
Loading file 521/25422: 1it [00:00, 1204.22it/s]
Loading file 522/25422: 1it [00:00, 1579.78it/s]
Loading file 523/25422: 1it [00:00, 1703.62it/s]
Loading file 524/2542

Loading file 671/25422: 1it [00:00, 1091.41it/s]
Loading file 672/25422: 1it [00:00, 2336.66it/s]
Loading file 673/25422: 1it [00:00, 1306.64it/s]
Loading file 674/25422: 1it [00:00, 2179.99it/s]
Loading file 675/25422: 1it [00:00, 2310.91it/s]
Loading file 676/25422: 1it [00:00, 969.33it/s]
Loading file 677/25422: 1it [00:00, 1345.19it/s]
Loading file 678/25422: 1it [00:00, 1599.05it/s]
Loading file 679/25422: 1it [00:00, 2172.09it/s]
Loading file 680/25422: 1it [00:00, 1343.47it/s]
Loading file 681/25422: 1it [00:00, 1531.33it/s]
Loading file 682/25422: 1it [00:00, 2279.51it/s]
Loading file 683/25422: 1it [00:00, 1640.96it/s]
Loading file 684/25422: 1it [00:00, 1572.08it/s]
Loading file 685/25422: 1it [00:00, 1504.41it/s]
Loading file 686/25422: 1it [00:00, 1367.11it/s]
Loading file 687/25422: 1it [00:00, 1757.14it/s]
Loading file 688/25422: 1it [00:00, 1327.31it/s]
Loading file 689/25422: 1it [00:00, 2164.24it/s]
Loading file 690/25422: 1it [00:00, 1523.54it/s]
Loading file 691/2542

...

Loading file 5424/25422: 1it [00:00, 1229.64it/s]
Loading file 5425/25422: 1it [00:00, 1375.18it/s]
Loading file 5426/25422: 1it [00:00, 2067.18it/s]
Loading file 5427/25422: 1it [00:00, 1252.03it/s]
Loading file 5428/25422: 1it [00:00, 1646.12it/s]
Loading file 5429/25422: 1it [00:00, 1417.95it/s]
Loading file 5430/25422: 1it [00:00, 1290.56it/s]
Loading file 5431/25422: 1it [00:00, 1612.57it/s]
Loading file 5432/25422: 1it [00:00, 2162.01it/s]
Loading file 5433/25422: 1it [00:00, 1422.28it/s]
Loading file 5434/25422: 1it [00:00, 2146.52it/s]
Loading file 5435/25422: 1it [00:00, 2098.20it/s]
Loading file 5436/25422: 1it [00:00, 1111.37it/s]
Loading file 5437/25422: 1it [00:00, 2078.45it/s]
Loading file 5438/25422: 1it [00:00, 1300.16it/s]
Loading file 5439/25422: 1it [00:00, 1999.19it/s]
Loading file 5440/25422: 1it [00:00, 2154.24it/s]
Loading file 5441/25422: 1it [00:00, 2122.62it/s]
Loading file 5442/25422: 1it [00:00, 1464.49it/s]
Loading file 5443/25422: 1it [00:00, 2144.33it/s]


Loading file 5587/25422: 1it [00:00, 1437.88it/s]
Loading file 5588/25422: 1it [00:00, 1645.47it/s]
Loading file 5589/25422: 1it [00:00, 1203.88it/s]
Loading file 5590/25422: 1it [00:00, 1677.72it/s]
Loading file 5591/25422: 1it [00:00, 1381.07it/s]
Loading file 5592/25422: 1it [00:00, 1225.69it/s]
Loading file 5593/25422: 1it [00:00, 1556.91it/s]
Loading file 5594/25422: 1it [00:00, 1437.88it/s]
Loading file 5595/25422: 1it [00:00, 1440.85it/s]
Loading file 5596/25422: 1it [00:00, 1680.41it/s]
Loading file 5597/25422: 1it [00:00, 1498.50it/s]
Loading file 5598/25422: 1it [00:00, 610.52it/s]
Loading file 5599/25422: 1it [00:00, 1708.47it/s]
Loading file 5600/25422: 1it [00:00, 2086.72it/s]
Loading file 5601/25422: 1it [00:00, 1446.31it/s]
Loading file 5602/25422: 1it [00:00, 1150.07it/s]
Loading file 5603/25422: 1it [00:00, 1420.35it/s]
Loading file 5604/25422: 1it [00:00, 1278.75it/s]
Loading file 5605/25422: 1it [00:00, 1804.00it/s]
Loading file 5606/25422: 1it [00:00, 1384.26it/s]
L

Loading file 5751/25422: 1it [00:00, 1300.96it/s]
Loading file 5752/25422: 1it [00:00, 1276.80it/s]
Loading file 5753/25422: 1it [00:00, 1471.17it/s]
Loading file 5754/25422: 1it [00:00, 1443.33it/s]
Loading file 5755/25422: 1it [00:00, 1527.98it/s]
Loading file 5756/25422: 1it [00:00, 1264.49it/s]
Loading file 5757/25422: 1it [00:00, 1384.72it/s]
Loading file 5758/25422: 1it [00:00, 1370.69it/s]
Loading file 5759/25422: 1it [00:00, 989.69it/s]
Loading file 5760/25422: 1it [00:00, 1280.31it/s]
Loading file 5761/25422: 1it [00:00, 1382.43it/s]
Loading file 5762/25422: 1it [00:00, 1836.39it/s]
Loading file 5763/25422: 1it [00:00, 1388.84it/s]
Loading file 5764/25422: 1it [00:00, 1242.76it/s]
Loading file 5765/25422: 1it [00:00, 1311.54it/s]
Loading file 5766/25422: 1it [00:00, 1313.18it/s]
Loading file 5767/25422: 1it [00:00, 1395.78it/s]
Loading file 5768/25422: 1it [00:00, 1474.79it/s]
Loading file 5769/25422: 1it [00:00, 1521.33it/s]
Loading file 5770/25422: 1it [00:00, 1906.50it/s]
L

Loading file 5915/25422: 1it [00:00, 1283.45it/s]
Loading file 5916/25422: 1it [00:00, 1197.00it/s]
Loading file 5917/25422: 1it [00:00, 1264.11it/s]
Loading file 5918/25422: 1it [00:00, 1001.51it/s]
Loading file 5919/25422: 1it [00:00, 1090.28it/s]
Loading file 5920/25422: 1it [00:00, 1163.15it/s]
Loading file 5921/25422: 1it [00:00, 1251.66it/s]
Loading file 5922/25422: 1it [00:00, 1414.61it/s]
Loading file 5923/25422: 1it [00:00, 1163.79it/s]
Loading file 5924/25422: 1it [00:00, 1269.85it/s]
Loading file 5925/25422: 1it [00:00, 1539.76it/s]
Loading file 5926/25422: 1it [00:00, 1369.35it/s]
Loading file 5927/25422: 1it [00:00, 983.89it/s]
Loading file 5928/25422: 1it [00:00, 1027.51it/s]
Loading file 5929/25422: 1it [00:00, 1630.76it/s]
Loading file 5930/25422: 1it [00:00, 1374.28it/s]
Loading file 5931/25422: 1it [00:00, 1148.50it/s]
Loading file 5932/25422: 1it [00:00, 1356.06it/s]
Loading file 5933/25422: 1it [00:00, 1328.15it/s]
Loading file 5934/25422: 1it [00:00, 1069.43it/s]
L

Loading file 6079/25422: 1it [00:00, 1906.50it/s]
Loading file 6080/25422: 1it [00:00, 1328.99it/s]
Loading file 6081/25422: 1it [00:00, 1223.19it/s]
Loading file 6082/25422: 1it [00:00, 1530.21it/s]
Loading file 6083/25422: 1it [00:00, 1494.23it/s]
Loading file 6084/25422: 1it [00:00, 1496.90it/s]
Loading file 6085/25422: 1it [00:00, 1390.22it/s]
Loading file 6086/25422: 1it [00:00, 1630.76it/s]
Loading file 6087/25422: 1it [00:00, 1565.04it/s]
Loading file 6088/25422: 1it [00:00, 475.44it/s]
Loading file 6089/25422: 1it [00:00, 1535.81it/s]
Loading file 6090/25422: 1it [00:00, 1727.47it/s]
Loading file 6091/25422: 1it [00:00, 1869.95it/s]
Loading file 6092/25422: 1it [00:00, 2005.88it/s]
Loading file 6093/25422: 1it [00:00, 1869.95it/s]
Loading file 6094/25422: 1it [00:00, 1399.03it/s]
Loading file 6095/25422: 1it [00:00, 1287.39it/s]
Loading file 6096/25422: 1it [00:00, 1814.15it/s]
Loading file 6097/25422: 1it [00:00, 2040.03it/s]
Loading file 6098/25422: 1it [00:00, 1972.86it/s]
L

In [ ]:
property_map = {
    'energy-forces-stress': [{
        # ColabFit name: {'field': ASE field name, 'units': str}
        'energy': {'field': 'energy', 'units': 'eV'},
        'forces': {'field': 'forces', 'units': 'eV/Ang'},
        'stress': {'field': 'stress', 'units': 'GPa'},
        'per-atom': {'field': 'per-atom', 'units': None},
        
        '_settings': {
            '_method': 'VASP',
            '_description': 'energy/forces/stresses',
            '_files': None,
            '_labels': ['PBE', 'GGA']
        }
    }]
}

In [ ]:
ids = list(client.insert_data(
    configurations,
    property_map=property_map,
    generator=False,
    verbose=True
))

all_co_ids, all_pr_ids = list(zip(*ids))

In [ ]:
len(set(all_co_ids))

In [ ]:
len(set(all_pr_ids))

Note: this dataset has multiple properties that point to the same configurations, but have different property values

In [ ]:
client.configurations.count_documents({'_id': {'$in': all_co_ids}, 'relationships.properties.1': {'$exists': True}})

In [ ]:
duplicate_pr_ids = client.configurations.find_one(
    {'_id': {'$in': all_co_ids}, 'relationships.properties.1': {'$exists': True}},
    {'relationships.properties'}
)['relationships']['properties']
duplicate_pr_ids

In [ ]:
pr0 = client.properties.find_one({'_id': duplicate_pr_ids[0]})
pr1 = client.properties.find_one({'_id': duplicate_pr_ids[1]})

In [ ]:
pr0['energy-forces-stress']['energy']['source-value']

In [ ]:
pr1['energy-forces-stress']['energy']['source-value']

In [ ]:
configuration_set_regexes = {
    '.*':
        'W-Be configurations for studying plasma material interactions in fusion reactors',
    '(001|010|100)FreeSurf':
        'Be [001], [010], and [100] surfaces',
    'DFTMD_1000K':
        'AIMD sampling of Be at 1000K',
    'DFTMD_300K':
        'AIMD sampling of Be at 300K',
    'DFT_MD_1000K':
        'AIMD sampling of W-Be at 1000K',
    'DFT_MD_300K':
        'AIMD sampling of W-Be at 300K',
    'Divacancy':
        'divacancy defects in pure W',
    '^EOS':
        'Energy vs. volume EOS configurations for W-Be',
    'EOS_(BCC|FCC|HCP)':
        'Energy vs. volume EOS configurations for Be in various crystal structures',
    'EOS_Data':
        'Energy vs. volume configurations for W',
    'Elast_(BCC|FCC|HCP)_(Shear|Vol)':
        'BCC, FCC, and HCP Be with shear or volumetric strains',
    'ElasticDeform_(Shear|Vol)':
        'W-Be in various crystal structures with shear and volumetric strains',
    'Liquids':
        'Liquid Be',
    'StackFaults':
        'Be stacking faults',
    'WSurface_BeAdhesion':
        'Be adhesion onto W surfaces',
    'dislocation_quadrupole':
        'W dislocation quadrupole configurations',
    '^gamma_surface/':
        'W gamma surface configurations',
    'gamma_surface_vacancy':
        'W gamma surface configurations',
    'md_bulk':
        'AIMD sampling of bulk W',
    '^surface':
        'pure W surfaces',
    '^vacancy':
        'bulk W with vacancy defects',
}

cs_ids = []

for i, (regex, desc) in enumerate(configuration_set_regexes.items()):
    co_ids = client.get_data(
        'configurations',
        fields='_id',
        query={'_id': {'$in': all_co_ids}, 'names': {'$regex': regex}},
        ravel=True
    ).tolist()

    print(f'Configuration set {i}', f'({regex}):'.rjust(35), f'{len(co_ids)}'.rjust(7))

    cs_id = client.insert_configuration_set(co_ids, description=desc)

    cs_ids.append(cs_id)

In [ ]:
ds_id = client.insert_dataset(
    cs_ids=cs_ids,
    pr_ids=all_pr_ids,
    name='WBe_PRB2019',
    authors=[
        'M. A. Wood', 'M. A. Cusentino', 'B. D. Wirth', 'A. P. Thompson'
    ],
    links=[
        'https://journals.aps.org/prb/abstract/10.1103/PhysRevB.99.184305',
        'https://github.com/FitSNAP/FitSNAP/tree/master/examples/WBe_PRB2019'
    ],
    description='This data set was originally used to generate a '\
    'multi-component linear SNAP potential for tungsten and beryllium as '\
    'published in Wood, M. A., et. al. Phys. Rev. B 99 (2019) 184305. This '\
    'data set was developed for the purpose of studying plasma material '\
    'interactions in fusion reactors.',
    resync=True,
    verbose=True,
)
ds_id

In [ ]:
configuration_label_regexes = {
    'FreeSurf':
        'surface',
    'surface':
        'surface',
    'BCC':
        'bcc',
    'DFT(_)?MD':
        'aimd',
    'Divacancy':
        'divacancy',
    'EOS':
        'eos',
    'Elast':
        'elastic',
    'Liquids':
        'liquid',
    'StackFaults':
        'stacking_fault',
    'dislocation':
        'dislocation',
    'gamma_surface':
        'gamma_surface',
    'md_bulk':
        'aimd',
}

for regex, labels in configuration_label_regexes.items():
    client.apply_labels(
        dataset_id=ds_id,
        collection_name='configurations',
        query={'_id': {'$in': all_co_ids}, 'names': {'$regex': regex}},
        labels=labels,
        verbose=True
    )

In [ ]:
ds_id = '5579011345901941240'
dataset = client.get_dataset(ds_id, resync=True, verbose=True)['dataset']

for k,v in dataset.aggregated_info.items():
    print(k,v)

In [ ]:
dataset.aggregated_info['property_fields']

In [ ]:
fig = client.plot_histograms(dataset.aggregated_info['property_fields'], ids=dataset.property_ids, yscale='log')

In [ ]:
client.dataset_to_markdown(
    ds_id=ds_id,
    base_folder='/colabfit/markdown/'+dataset.name,
    html_file_name='README.md',
    data_format='mongo',
    data_file_name=None,
    yscale='log'
)